<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/getScreenerData.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import openpyxl

from google.colab import drive
drive.mount('/content/drive')

from google.colab import data_table

#mypf = pd.read_csv('/content/drive/My Drive/data/stocks/Try-Hyperlink.xlsx')


def _get_link_if_exists(cell) -> str | None:
    try:
        return cell.hyperlink.target
    except AttributeError:
        return None


def extract_hyperlinks_from_xlsx(
    file_name: str, sheet_name: str, columns_to_parse: list[str], row_header: int = 1
) -> pd.DataFrame:
    df = pd.read_excel(file_name, sheet_name)
    ws = openpyxl.load_workbook(file_name)[sheet_name]
    for column in columns_to_parse:
        row_offset = row_header + 1
        column_index = list(df.columns).index(column) + 1
        df[column] = [
            _get_link_if_exists(ws.cell(row=row_offset + i, column=column_index))
            for i in range(len(df[column]))
        ]
    return df

df_data = extract_hyperlinks_from_xlsx('/content/drive/My Drive/quantvesting/data/myScreenerDB.xlsx','data', ['Name'])


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
df_data

,Name,CMP Rs.,ATH%,P/E,EPS 12M Rs.,CMP / BV,Mar Cap Rs.Cr.,ROCE %,ROE %,Sales growth %,Profit growth %,5Yrs PE,ROCE 5Yr %,ROE 5Yr %,Debt / Eq,NP 12M Rs.Cr.,CFO to EBITDA %,CapType
0,https://www.screener.in/company/ISEC/consolida...,896.2,2.93,15.01,59.70,5.38,29148.63,18.19,41.55,25.40,14.52,14.95,24.00,50.22,3.89,1941.45,-39.87,SC
1,https://www.screener.in/company/MEDANTA/consol...,1201.1,26.04,62.23,17.92,10.27,32260.03,NaN,NaN,12.74,8.44,61.56,13.88,NaN,0.22,481.32,65.24,SC
2,https://www.screener.in/company/INDIGO/,5562.5,1.56,35.39,157.31,57.56,214956.90,24.53,NaN,17.22,-15.52,23.36,1.82,NaN,15.86,6073.80,119.94,SC


In [3]:
df_data.columns

Index(['Name', 'CMP Rs.', 'ATH%', 'P/E', 'EPS 12M Rs.', 'CMP / BV',
       'Mar Cap Rs.Cr. ', 'ROCE %', 'ROE %', 'Sales growth %',
       'Profit growth %', '5Yrs PE', 'ROCE 5Yr %', 'ROE 5Yr %', 'Debt / Eq',
       'NP 12M Rs.Cr.', 'CFO to EBITDA %', 'CapType'],
      dtype='object')

In [4]:
df_data['Symbol'] = df_data['Name'].str.split('company/').str[1].str.split('/').str[0]
df_data['Latest']  = 1
df_data.columns = ['Name', 'CMP', 'ATH%', 'PE', 'EPS', 'PB', 'MCap', 'ROCE%', 'ROE%', 'Sales_Grwth%', 'Profit_Grwth%', 'MedPE', 'ROCE_5Yr%',
       'ROE_5Yr%', 'Debt2EqR', 'PAT_12M', 'CFO_2_EBITDA%', 'CapType', 'Symbol', 'Latest']

In [5]:
df_data

,Name,CMP,ATH%,PE,EPS,PB,MCap,ROCE%,ROE%,Sales_Grwth%,Profit_Grwth%,MedPE,ROCE_5Yr%,ROE_5Yr%,Debt2EqR,PAT_12M,CFO_2_EBITDA%,CapType,Symbol,Latest
0,https://www.screener.in/company/ISEC/consolida...,896.2,2.93,15.01,59.70,5.38,29148.63,18.19,41.55,25.40,14.52,14.95,24.00,50.22,3.89,1941.45,-39.87,SC,ISEC,1
1,https://www.screener.in/company/MEDANTA/consol...,1201.1,26.04,62.23,17.92,10.27,32260.03,NaN,NaN,12.74,8.44,61.56,13.88,NaN,0.22,481.32,65.24,SC,MEDANTA,1
2,https://www.screener.in/company/INDIGO/,5562.5,1.56,35.39,157.31,57.56,214956.90,24.53,NaN,17.22,-15.52,23.36,1.82,NaN,15.86,6073.80,119.94,SC,INDIGO,1


In [6]:
df_myscreener = pd.read_csv('/content/drive/My Drive/quantvesting/data/myScreenerDB.csv')
df_tmp = df_myscreener[~df_myscreener['Symbol'].isin(df_data['Symbol'])]
df_myscreener = pd.concat([df_data, df_tmp], ignore_index=True)

In [7]:
df_myscreener = df_myscreener.sort_values(by = 'MCap', ascending=False)
df_myscreener['Category'] = 'SC'  # Default category
df_myscreener.iloc[:100, df_myscreener.columns.get_loc('Category')] = 'LC'
df_myscreener.iloc[100:250, df_myscreener.columns.get_loc('Category')] = 'MC'

In [11]:
#df_myscreener.to_csv('/content/drive/My Drive/quantvesting/data/myScreenerDB.csv', index=False)

In [9]:
cols = ['Symbol','PE', 'EPS','MCap','ROCE%','ROE%','MedPE','ROCE_5Yr%','ROE_5Yr%','Debt2EqR','Sales_Grwth%', 'Profit_Grwth%','CFO_2_EBITDA%','PAT_12M','PB','CapType']
df = df_myscreener[cols]

In [10]:
df_tmp = df[(df['PE'] < df['MedPE']) & (df['Debt2EqR'] < .25) & (df['Sales_Grwth%'] > 15) & (df['Profit_Grwth%'] > 25)]
data_table.DataTable(df_tmp, include_index=False)

,Symbol,PE,EPS,MCap,ROCE%,ROE%,MedPE,ROCE_5Yr%,ROE_5Yr%,Debt2EqR,Sales_Grwth%,Profit_Grwth%,CFO_2_EBITDA%,PAT_12M,PB,CapType
52,VBL,60.94,8.46,169651.29,24.85,22.49,66.71,21.76,24.98,0.17,29.09,28.64,68.53,2817.66,10.21,LC
53,DLF,44.43,16.18,166761.47,5.74,6.95,56.92,4.64,4.27,0.11,21.79,58.15,97.78,4004.44,4.15,LC
108,ICICIGI,36.56,50.60,91713.34,24.86,18.80,43.93,22.88,17.90,0.00,16.95,30.61,34.54,2508.29,6.33,MC
111,ZYDUSLIFE,19.71,45.09,89333.45,22.34,20.67,23.33,15.20,19.79,0.01,16.94,38.62,57.08,4675.00,4.10,MC
125,WAAREEENER,44.87,65.00,83918.30,35.12,27.57,47.48,33.44,29.97,0.13,26.73,89.26,101.12,1928.13,8.85,MC
127,POLICYBZR,255.54,7.69,80654.76,5.90,5.13,572.76,-6.21,-4.94,0.05,44.78,390.25,-36.51,353.16,12.54,MC
131,SUZLON,65.25,0.85,76368.28,24.93,28.77,102.63,5.64,NaN,0.06,54.24,136.36,7.39,1144.77,16.97,MC
154,OBEROIRLTY,27.39,61.21,60947.01,18.16,15.06,28.08,14.14,13.16,0.21,17.58,26.64,65.56,2225.51,3.88,MC
208,VOLTAS,50.03,25.43,41737.80,17.86,13.53,71.47,12.79,8.67,0.14,23.49,236.04,-17.05,834.28,6.41,MC
214,GVT&D,82.05,19.06,40048.89,23.14,15.66,157.11,3.87,-2.79,0.03,37.06,450.69,146.23,488.13,29.07,MC
